In [ ]:
from qdrant_client import QdrantClient
from fastembed import TextEmbedding

In [2]:
client = QdrantClient("http://localhost:6333")

In [17]:
emb_model = TextEmbedding("jinaai/jina-embeddings-v2-small-en")

embeddings_generator = emb_model.query_embed("I just discovered the course. Can I join now?")
embeddings = list(embeddings_generator)[0]
len(embeddings)

512

In [18]:
embeddings.min()

np.float64(-0.11726373885183883)

In [12]:
import numpy as np
np.linalg.norm(embeddings)

np.float64(1.0)

In [19]:
doc = 'Can I still join the course after the start date?'

embeddings_2 = list(emb_model.query_embed(doc))[0]
embeddings.dot(embeddings_2)

np.float64(0.9008528895674548)

In [14]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [31]:
emb_arrays_list = list(emb_model.embed(
    [doc["text"] for doc in documents]
))

emb_matrix = np.stack(emb_arrays_list)

In [38]:
scores = emb_matrix.dot(embeddings)
scores

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [39]:
np.argsort(-scores)[0]

np.int64(1)

In [40]:
documents_full_text = []
for doc in documents:
    full_text = doc["question"] + " " + doc["text"]
    documents_full_text.append(full_text)

emb_arrays_list2 = list(emb_model.embed(
    documents_full_text
))

emb_matrix2 = np.stack(emb_arrays_list2)

In [43]:
scores2 = emb_matrix2.dot(embeddings)
scores2

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

In [44]:
np.argsort(-scores2)[0]

np.int64(0)

In [46]:
dims = [model["dim"] for model in TextEmbedding.list_supported_models()]
min(dims)

384

In [47]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [50]:
bge_emb_model = TextEmbedding("BAAI/bge-small-en")

documents_full_text = []
for doc in documents:
    full_text = doc["question"] + " " + doc["text"]
    documents_full_text.append(full_text)

emb_arrays_list3 = list(bge_emb_model.embed(
    documents_full_text
))

emb_matrix3 = np.stack(emb_arrays_list3)

In [55]:
bge_embeddings = list(bge_emb_model.query_embed("I just discovered the course. Can I join now?"))[0]

scores3 = emb_matrix3.dot(bge_embeddings)
max(scores3)

np.float32(0.8703172)